In [2]:
import sys
from astropy.table import Table

In [3]:
t = Table([['<a href="http://coucou.fr">foobar</a>'], [1]])
t

col0,col1
str37,int64
"<a href=""http://coucou.fr"">foobar</a>",1


## Problem : link is escaped

In [4]:
t.write(sys.stdout, format='html')

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html;charset=UTF-8" http-equiv="Content-type"/>
 </head>
 <body>
  <table>
   <thead>
    <tr>
     <th>col0</th>
     <th>col1</th>
    </tr>
   </thead>
   <tr>
    <td>&lt;a href="http://coucou.fr"&gt;foobar&lt;/a&gt;</td>
    <td>1</td>
   </tr>
  </table>
 </body>
</html>



## The HTML writer has an option for this

In [5]:
t.write(sys.stdout, format='html', htmldict=dict(raw_html_cols=['col0']))

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html;charset=UTF-8" http-equiv="Content-type"/>
 </head>
 <body>
  <table>
   <thead>
    <tr>
     <th>col0</th>
     <th>col1</th>
    </tr>
   </thead>
   <tr>
    <td><a href="http://coucou.fr">foobar</a></td>
    <td>1</td>
   </tr>
  </table>
 </body>
</html>



## But not the jsviewer writer ...

In [6]:
t.write(sys.stdout, format='jsviewer', htmldict=dict(raw_html_cols=['col0']))

TypeError: write_table_jsviewer() got an unexpected keyword argument 'htmldict'

## Redefine the jsviewer writer to fix that

In [7]:
from astropy.table.jsviewer import DEFAULT_CSS, JSViewer

def write_table_jsviewer(table, filename, table_id=None, max_lines=5000,
                         table_class="display compact", jskwargs=None,
                         css=DEFAULT_CSS, html_kwargs=None):
    if table_id is None:
        table_id = 'table{id}'.format(id=id(table))

    jskwargs = jskwargs or {}
    jsv = JSViewer(**jskwargs)

    sortable_columns = [i for i, col in enumerate(table.columns.values())
                        if col.dtype.kind in 'iufc']
    htmldict = {
        'table_id': table_id,
        'table_class': table_class,
        'css': css,
        'cssfiles': jsv.css_urls,
        'jsfiles': jsv.jquery_urls,
        'js': jsv.html_js(table_id=table_id, sort_columns=sortable_columns)
    }
    if html_kwargs:
        htmldict.update(html_kwargs)

    if max_lines < len(table):
        table = table[:max_lines]
    table.write(filename, format='html', htmldict=htmldict)

In [8]:
write_table_jsviewer(t, sys.stdout, html_kwargs=dict(raw_html_cols=['col0']))

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html;charset=UTF-8" http-equiv="Content-type"/>
  <style>
body {font-family: sans-serif;}
table.dataTable {width: auto !important; margin: 0 !important;}
.dataTables_filter, .dataTables_paginate {float: left !important; margin-left:1em}
  </style>
  <link href="https://cdn.datatables.net/1.10.12/css/jquery.dataTables.css" rel="stylesheet" type="text/css"/>
  <script src="https://code.jquery.com/jquery-3.1.1.min.js">
  </script>
  <script src="https://cdn.datatables.net/1.10.12/js/jquery.dataTables.min.js">
  </script>
 </head>
 <body>
  <script>
var astropy_sort_num = function(a, b) {
    var a_num = parseFloat(a);
    var b_num = parseFloat(b);

    if (isNaN(a_num) && isNaN(b_num))
        return ((a < b) ? -1 : ((a > b) ? 1 : 0));
    else if (!isNaN(a_num) && !isNaN(b_num))
        return ((a_num < b_num) ? -1 : ((a_num > b_num) ? 1 : 0));
    else
        return isNaN(a_num) ? -1 : 1;
}

jQuery.extend( jQuery.fn.dataTa